# <font color='SteelBlue'>Structural Pattern Matching</font>
* 是Python 3.10最重大的新增功能。
* 類似C-like系列語言的switch-case但<font color='tomato' size='+1'>更強大</font>(truly awesome)。
* 筆者建議盡快熟悉使用。

註：建議將JupyterLab的Theme從預設的Light改為Dark，閱讀效果會較佳。

In [2]:
import sys
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
print("InteractiveShell set.")
print(sys.version)

InteractiveShell set.
3.10.6 (main, Nov  2 2022, 18:53:38) [GCC 11.3.0]


## <font color='LightSalmon'><font color='LightSalmon'>多重判斷時Python的傳統寫法</font>
* 3.9版以前，Python沒有switch-case，多重判斷只能if-elif-else。筆者覺得長相不佳，認為是Python語法上少數暗點之一。是否為暗點當然見仁見智，總之個人不大喜歡就是。
* 以下用一個最簡單的例子複習Python傳統的if-elif-else語法。筆者對此有意見，原因是'=='右邊的<font color='tomato' size='+1'>「定數」(literal)</font>無法對齊。

In [2]:
http_code = "418"

if http_code == "200":  # 這個'200'和下面的'401', '418'等地位相同卻對不齊。
    print("OK")
elif http_code == "401" or http_code == "403" or http_code == "404":
    print("Not Found")
elif http_code == "418":
    print("I'm a teapot")
elif http_code == "500":
    print("Internal Server Error")
else:
    print("Code not found")

I'm a teapot


## <font color='LightSalmon'><font color='LightSalmon'>Structural Pattern Matching語法</font>
* 終於終於，Python 3.10版可以用類似switch-case的結構了，這個新語法稱為<font color='tomato' size='+1'>Structural Pattern Matching</font>。
* 本文其後簡稱作Pattern Matching或乾脆叫Matching。
* 以下用Pattern Matching改寫樓上的if-elif-else。結構簡潔，定數漂亮對齊。

In [ ]:
http_code = '404'

match http_code:
    # match和case之間不可以有任何指令。可試打開下面註解掉的任何一行。
    # print('Pattern Matching test.')
    # if True:
    #     pass
    case '200':                  # '200'和下面的'401', '418'等對齊，整齊多了。
        print ('OK')
    # 2個cases之間也不能有指令。
    # name = "Alex"
    case '401' | '403' | '404':  # 可以用'|'表示「或」。
        print('Not allowed')
    case '418':
        print("I'm a teapot")
    case '500':
        print('Internal Server Error')
    case _:                      # 相當於if-elif-else中的else。
        print("Something's wrong with the internet")

## <font color='LightSalmon'>Cool，但談不上啥特異功能吔</font>
* 有人也許會說，這個Pattern Matching很漂亮，可只是語法上的改進或語法甜頭(syntactic sugar)而已，並非「新增功能」。
* 沒錯，Pattern Matching能耐如僅及於此，的確不算新增功能。不過上面這段只是它的基本入門款，牛刀小試耳。
* 此功能關卡堡壘不少，必須耐心逐個攻破，方得全貌。
* 在下將步步為營，謹慎求證，不會重蹈[「市場花園行動」](https://zh.wikipedia.org/wiki/%E5%B8%82%E5%A0%B4%E8%8A%B1%E5%9C%92%E8%A1%8C%E5%8B%95)覆轍。<font color='tomato' size='+1'>橋雖遠，終有到達一天</font>。
* 搶占Pattern Matching灘頭堡的[「諾曼第登陸」](https://zh.wikipedia.org/wiki/%E8%AF%BA%E6%9B%BC%E5%BA%95%E6%88%98%E5%BD%B9)開始：

## <font color='LightSalmon'>Pattern Matching-0：match和case非保留字</font></font>
* 3.10版雖然新增了match-case的Matching結構，但match和case這兩個字[只是soft keywords，不是hard keywords](https://www.python.org/dev/peps/pep-0622/)，意思是這2個key words在3.10版<font color='tomato' size='+1'>並未增列為保留字</font>(reserved words)。這個措施使得3.10版和之前的版本相容，各位可以放心使用match和case作為變數。
* Pattern Matching結構本身match和case也不是變數。

In [3]:
from pprint import pprint

match = 'I am a match'
# case = 'I am a case'

match match:   # 前面的match是Pattern Matching的關鍵字，後面的match則是變數。
# match case:
    case 'I am a match':
        print('matches case 0')
    case 'I am a case':
        print('matches case 1')
    case _:
        print('no match')


if "match" in globals():
    print(f'match: {match}')
if "case" in locals():   # 用globals()和locals()判斷好像都行。不確定。
    print(f'case: {case}')
# 經過Pattern Matching之後，match和case的值不變。

matches case 0
match: I am a match


## <font color='LightSalmon'>Pattern Matching-1：pattern內可以有變數</font>
* Pattern Matching就是可以用'pattern'來比對。
* 意思是：case的後面除了「定數」外，還可以是個pattern。
* pattern內可以有<font color='tomato' size='+1'>變數</font>，這是Pattern Matching比switch-case更為強大的原因之一。
* pattern比對順序由上至下，而且最多只有1個匹配。前面的pattern匹配到，就自動跳離match結構，不再往後比對了。因此case內不必也不能break。

In [ ]:
dict_to_match = {'name': 'Becca', 'age': 28}
# dict_to_match = {'name': 'Becca', 'age': 6}
match dict_to_match:      # 所謂Pattern Matching，就是可以用'pattern'來比對。
    # case後面的就是pattern。
    case {'name': 'Alex', 'age': 38}:   # ．此pattern比對'Alex'和38。
        print('match case 0')           #   匹配到會自動跳離，不須也不能寫break。
    case {'name': name, 'age': age}:    # ．以變數name和age比對任何名稱和年齡。
        print('match case 1')           #   value的變數名稱不一定要和key名稱相同。
    case {'name': name1, 'age': age1}:  # ．本pattern和case 1的pattern完全相同。因
        print('match case 2')           #   case 1較優先，所以本case永遠不會有匹配。
    case _:
        print('no match')

## <font color='LightSalmon'>Pattern Matching-2：不一定要包含dict的全部keys</font>
* 要比對的物件(即match後面的變數)如果是dict，<font color='tomato' size='+1'>pattern不一定要包含該dict所有的keys</font>。
* 可少卻不可多。如果pattern多出一個原dict沒有的key，就無法匹配。
* dict和pattern的keys順序可以不一樣。

In [ ]:
def match(obj):
    match_obj = 'no match'
    match obj:
        case {'id': id,      # 本pattern匹配任何有這些keys的dict。
              'info': {'name': name, 'age': age}}:  # pattern不須包括dict的全部keys。
            match_obj = {'id': id, 'info': {'name': name, 'age': age}}
            print(r"match pattern    : {'id': id, 'info': {'name': name, 'age': age}}")
            print(f'matches case 0   : {match_obj}' '\n')
        case {'id': id,        # 比對dict時，pattern的key可少卻不可多。
              'name': name,    # 這個code cell傳入本函數的person，全都沒有一個
              'edu': edu,      # 名叫'edu'的key，所以本pattern無法匹配任何person。
              'bmi': bmi}:
            match_obj = {'id': id, 'name': name, 'edu': edu, 'bmi': bmi}
            print(r"match pattern    : {'id': id, 'name': name, 'edu': edu, 'bmi': bmi}")
            print(f'matches case 1   : {match_obj}' '\n')
        case {'name': {'first': first, 'last': last,},
              'id': id}:       # keys的順序不一樣，亦匹配第3個person。
            match_obj = {'name': {'first': first, 'last': last}, 'id': id}
            print(r"match pattern    : {'name': {'first': first, 'last': last}, 'id': id}")
            print(f'matches case 2   : {match_obj}' '\n')
        case _:  # 以上皆非
            print(r'match pattern    : _')
            print(f'{match_obj}' '\n')
    return match_obj

# 這個persons內的元素是要被比對的物件。
persons = [
    {'id': 1, 'info': {'name': 'Alex', 'age': 49, 'hair': None}},
    {'id': 2, 'name': 'Lotus', 'gender': 'F', 'bmi': 21.25},
    {'id': 3, 'name': {'last': '藍', 'first': '可兒'}, 'is_veg': False},
]

for person in persons:
    print(f'obj to be matched: {person}      type: {type(person)}')
    result = match(person)

## <font color='LightSalmon'>Pattern Matching-3：list和tuple意義相同</font>
* 比對時list和tuple意義完全相同，都是<font color='tomato' size='+1'>sequence(序列)</font>。
* 所謂sequence，可粗略理解成「可使用索引以及切片」的型別，如list, tuple和str。
* Pattern Matching機制為了避免產生某些錯誤，在<font color='tomato' size='+1'>match sequence時排除了str</font>。所以str不歸sequence管，是獨立比對的。

In [ ]:
def match(obj):
    match_obj = 'no match'
    match obj:
        case (item1, item2):
            match_obj = (item1, item2)
            print(r'match pattern    : (item1, item2)')
            print(f'matches case 0   : {match_obj}' '\n')
        case [item1, item2, item3]:
            match_obj = [item1, item2, item3]
            print(r'match pattern    : [item1, item2, item3]')
            print(f'matches case 1   : {match_obj}' '\n')
        case 'pi':
            match_obj = 'pi'
            print(r"match pattern    : 'pi'")
            print(f'matches case 2   : {match_obj}' '\n')
        case _:
            print(r'match pattern    : _')
            print(f'{match_obj}' '\n')
    return match_obj

objs = (
    [0, 1],                                       # 2-item list
    (True, False, None),                          # 3-item tuple
    [[0, 1, 1, 2, 3, 5], (2, 3, 5, 7, 11, 13)],   # 2-item list
    (2.71828,),                                   # 1-item tuple(Euler's number, e)
    ['春分', '夏至', '秋分', '冬至'],             # 4-item list
    (),                                           # 0-item tuple
    "['Isaac Newton', 'Albert Einstein']"         # str
)

for obj in objs:
    print(f'obj to be matched: {obj}      type: {type(obj)}')
    result = match(obj)

## <font color='LightSalmon'>Pattern Matching-4：支援萬用字元</font>
* Pattern Matching支援wildcards(萬用或百搭字元)比對，如<font color='tomato' size='+1'><b>[x, y, *rest]</b></font>或<font color='tomato' size='+1'><b>(x, y, *rest)</b></font>。意思是：這2個patterns要比對一個有最少有2個元素(最多不限)的sequence(list or tuple)。rest可以改用其他合法變數名稱如`[x, y, *others]`或者`(x, y, *others)`。
* <font color='tomato' size='+1'><b>*rest</b></font>其實就是Python著名的[unpacking秘技](https://www.learncodewithmike.com/2019/12/python-unpacking.html)的其中一部分。
* <font color='tomato' size='+1'><b>[x, y, *_]</b></font>或<font color='tomato' size='+1'><b>(x, y, *_)</b></font>則較為特殊。以筆者測試後的粗淺理解，這個寫法意思是比對一個最少有2個元素的sequence，可是第3個以後的元素卻不會unpack，或者說它也有unpack，但方式和一般不同。
* 至於dict的比對，也支援wildcard<font color='tomato' size='+2'><b>&#42;&#42;rest</b></font>，但不允許用<font color='tomato' size='+2'><b>**&#95;</b></font>。

In [ ]:
def match(obj):
    match_obj = 'no match'
    match obj:
        case (item1, item2):
            match_obj = (item1, item2)
            print(r'match pattern    : (item1, item2)')
            print(f'matches case 0   : {match_obj}' '\n')
        case [item1, item2, *others]:         # 請試將這個case區塊註解，再看結果。
            match_obj = [item1, item2, *others]
            print(r'match pattern    : [item1, item2, *others]')
            print(f'matches case 1   : {match_obj}' '\n')
        case (item1, item2, *_):
            match_obj = (item1, item2, _)     # 這裡'_'就好，不要'*_'再unpack一次。
            print(r'match pattern    : (item1, item2, *_)')
            print(f'matches case 2   : {match_obj}' '\n')
        case {"name": name, **other_pairs}:   # 用**other_pairs來百搭dict其他的pairs。
            match_obj = (name, other_pairs)
            print(r'match pattern    : {"name": name, **other_pairs}')
            print(f'matches case 3   : {match_obj}' '\n')
        # case {"name": name, **_}:             # invalid syntax
        #     match_obj = (name, _)
        #     print(f'matches case 4   : {match_obj}')
        #     print(r'match pattern    : {"name": name, **_}' '\n')
        case _:
            print(r'match pattern    : _')
            print(f'{match_obj}' '\n')
    return match_obj

objs = (
    ('Σ',),                      # 1-item tuple
    ['Σ', 'Φ'],                  # 2-item list
    ('Σ', 'Φ', 'Θ', 'Ψ', 'Ω'),   # 5-item tuple
    {'name': '王羲之', 'category': '書法', 'piece': '蘭亭集序'},
)

for obj in objs:
    print(f'obj to be matched: {obj}      type: {type(obj)}')
    result = match(obj)

## <font color='LightSalmon'>先岔開複習一下packing / unpacking</font>

In [ ]:
# def show_members(width, member1, member2, member3, member4, member5):
def show_members(width, *members):  # 將傳入的n個參數打包成一個名叫members的tuple。
    print("  --- within show_members() ---")
    print("  members: ", end="")
    for member in members:
        print(f"{member:{width}}", end="")


length = 12
# 將list的所有元素一次性賦予「同樣數量」的變數。
name1, name2, name3 = ["Lean", "Mirror", "Lotus"]
print("．將list的所有元素一次性賦予「同樣數量」的變數。注意：list有幾個元素，就要用幾個變數承接。")
print(f"  指令： name1, name2, name3 = ['Lean', 'Mirror', 'Lotus']")
print(f"  name1: {name1:{length}}name2: {name2:{length}}name3: {name3:{length}}")
print()

# Python慣例：用底線'_'代表don't care。
me, _, spouse = ("Alex", "Passion", "Marrianne")  # 'Passion'是小三？
print("．Python慣例用底線'_'代表don't care。")
print("  指令： me, _, spouse = ('Alex', 'Passion', 'Marrianne')   # 'Passion'是小三？")
print(f"  me: {me:{length}}spouse: {spouse}")
print()

# 剔頭去尾後，中間部分以'*'這個operator打包成others，others型別是list。
my_family = ("Alex", "Marrianne", "Rebecca", "Thomas", "Asing", "Audrey", "Cyrus")
me, *others, grandson = my_family  # 這裡有人會說成unpack，但我認為pack較易理解。
print("．先挑出sequence的前或後，其餘部分則「打包」(pack)成list。")
print(f"  my_family: {my_family}")
print("  指令： me, *others, grandson = my_family")
print(f"  me: {me:{length}}grandson: {grandson}")
print(f"  others: {others}      type: {type(others)}")
print()

rhesis = "當從實地建基柱，莫在浮沙築高樓。"
first, *middles, last = rhesis
print("．字串也是sequence。不過Pattern Matching中的sequence matching不包括str。")
print(f"  rhesis: {rhesis}")
print("  指令： first, *middles, last = rhesis")
print(f"  first: {first:{length}}last: {last}")
print(f"  middles: {middles}")
print()

print("．calling show_members(): show_members(length, *others)")
show_members(length, *others)  # 將others unpack為多個參數傳給show_members()。
# show_members(length, "Marrianne", "Rebecca", "Thomas", "Asing", "Audrey")

## <font color='LightSalmon'>Pattern Matching-5：設定變數型別</font>
* pattern中的變數如果沒有指定型別，會匹配任何型別物件。
* 如想指明型別，可用<font color='tomato' size='+1'><b>str(var)</b></font>, <font color='tomato' size='+1'><b>int(var)</b></font>, <font color='tomato' size='+1'><b>float(var)</b></font>等方式設定。

In [ ]:
def match(obj):
    match_obj = 'no match'
    match obj:
        case [str(item1), str(item2)]:  # 比對1個sequence，內有2個型別為str的元素。
            match_obj = [item1, item2]
            print(f'matches case 0   : {match_obj}')
            print(r'match pattern    : [str(item1), str(item2)]' '\n')
        case (int(num1), float(num2)):  # 比對1個sequence，2個元素，分別是int和float。
            match_obj = (num1, num2)
            print(f'matches case 1   : {match_obj}')
            print(r'match pattern    : (int(num1), float(num2))' '\n')
        case _:                         # 以上皆非
            print(match_obj)
            print('match pattern    : _' '\n')
            pass
    return match_obj

objs = [
    ('沈括', '張衡'),  # 華夏古代大科學家
    [1, 1644],         # 1644年中國有5個皇帝:崇禎、順治、李自成、張獻忠、南明弘光帝
    [0.0, 1.618],      # 黃金比例=1.618
    (0, 3.1415927),    # 南北朝時代祖沖之計算的Pi精確到小數點後7位，領先全球一千多年
    (1.41421, 6174)    # 畢達哥拉斯常數, 黑洞數
]

for obj in objs:
    print(f'obj to be matched: {obj}      type: {type(obj)}')
    result = match(obj)

## <font color='LightSalmon'>Pattern Matching-6：set的pattern比對</font>
依目前測試，發現：
* 比對set時，pattern不可以用<font color='tomato' size='+1'><b>{var1, var2, ...}</b></font>，而要用<font color='tomato' size='+1'><b>set()</b></font>或者<font color='tomato' size='+1'><b>set(var)</b></font>。
* <font color='tomato' size='+1'><b>set()</b></font>及<font color='tomato' size='+1'><b>set(var)</b></font>都可匹配任意數量元素的set。
* 但<font size='+1'><b>`set()`</b></font>的寫法因為沒有變數，無法抓到set內的元素。

In [ ]:
def match(obj):
    match_obj = 'no match'
    match obj:
        # 請試註解下面的case，看結果如何。
        # case set():               # 比對set要用set()，不可以{stuff, ...}。
        #     match_obj = set()     # 此pattern雖匹配任意的set，但抓不到set內元素。
        #     print(f'matches case 0   : {match_obj}')
        #     print('match pattern    : set()' '\n')
        # 上下兩個patterns都匹配任意元素的set，所以下面的pattern永遠沒有機會大展身手。
        case set(stuff):
            match_obj = stuff     # 設定變數，就可以抓到set的所有元素。
            print(f'matches case 1   : {match_obj}')
            print(r'match pattern    : set(stuff)' '\n')
        case _:
            print(match_obj)
            print('match pattern    : _' '\n')
            pass
    return match_obj

objs = (
    set(),
    {'Spagetti'},                                    # 意粉
    {'Spagetti', 'Macaroni'},                        # 意粉, 通粉
    {'Spagetti', 'Macaroni', 'Fusilli'},             # 意粉, 通粉, 螺旋粉
    {'Spagetti', 'Macaroni', 'Fusilli', 'Lasagna'},  # 意粉, 通粉, 螺旋粉, 千層麵
)

for obj in objs:
    print(f'obj to be matched: {obj}      type: {type(obj)}')
    result = match(obj)

## <font color='LightSalmon'>Pattern Matching-7：class也可比對</font>
* 以下用一個自訂類別<font color='tomato' size='+1'><b>Couple</b></font>示範class的比對。

In [ ]:
class Couple:
    def __init__(self, husband: str, wife: str):  # constructor(建構子)
        self.he: str = husband
        self.she: str = wife

    def __repr__(self):
        return f"Husband: {self.he}\tWife: {self.she}."

    def have_sex(self):   # method(方法)
        print('We are having sex...')
        return 'having sex'


def match(obj):
    match_obj = 'no match'
    match obj:
        case Couple(husband='Alex', wife='Lotus'):
            match_obj = ('Alex', 'Lotus')
            print(f'matches case 0   : {match_obj}')
            print("match pattern    : Couple(husband='Alex', wife='Lotus')" '\n')
        case Couple(husband='Arian', wife=str(wife)):
            match_obj = ('Arian', wife)
            print(f'matches case 1   : {match_obj}')
            print("match pattern    : Couple(husband='Arian', wife=wife)" '\n')
        # case Couple(he=str(husband), she='Diana'):
        # case Couple(she='Diana'):
        case Couple(have_sex='having sex'):  # TODO:
            # match_obj = ('Diana', )
            match_obj = ('having sex.......', )
            print(f'matches case 2   : {match_obj}')
            # print("match pattern    : Couple(husband=husband, wife='Lotus')" '\n')
            # print("match pattern    : Couple(she='Diana')" '\n')
            print("match pattern    : Couple(have_sex='having sex')" '\n')
        case Couple(husband=str(husband), wife=str(wife)):
            match_obj = (husband, wife)
            print(f'matches case 3   : {match_obj}')
            print('match pattern    : Couple(husband=husband, wife=wife)' '\n')
        case _:
            print(match_obj)
            print('match pattern    : _' '\n')
            pass
    return match_obj

couple1 = Couple('Arian', 'Brett')  # case 1
couple2 = Couple('Chris', 'Diana')  # case 2
couple3 = Couple('Ethan', 'Fiona')  # case 3
couple4 = Couple('Grant', 2021)     # no match
objs = (couple1, couple2, couple3, couple4)

for obj in objs:
    print(f'obj to be matched: {obj}      type: {type(obj)}')
    result = match(obj)

## <font color='LightSalmon'>Pattern Matching-8：Guard</font>
* pattern的後面可以加if判斷，這個if clause稱為<font color='tomato' size='+1'><b>guard</b></font>。
* guard的判斷結果為False時，會往下一個case走。
* case後面的變數會先抓值，然後交給guard運用。

In [ ]:
def match(obj):
    match_obj = 'no match'
    match obj:
        case [str(item1), str(item2)] if item1 == item2:  # 2個字串元素必須相同。
            match_obj = [item1, item2]
            print(f'matches case 0   : {match_obj}')
            print(r'match pattern    : str(item1), str(item2)] if item1 == item2' '\n')
        case (int(n1), int(n2), int(n3), int(n4), int(n5), int(n6)) \
              if n1 + n2 == n3 and n2 + n3 == n4 and n3 + n4 == n5 and n4 + n5 == n6:
            match_obj = (n1, n2, n3, n4, n5, n6)
            print(f'matches case 1   : {match_obj}')
            print(r'match pattern    : (int(n1), int(n2), int(n3), int(n4), int(n5), int(n6)) if n1 + n2 == n3 and n2 + n3 == n4 and n3 + n4 == n5 and n4 + n5 == n6' '\n')
        case (int(n1), int(n2), int(n3), int(n4), int(n5)) if [i**2 for i in range(1, 6)]:
        # case (int(n1), int(n2), int(n3), int(n4), int(n5)) if [1, 4, 9, 16, 25]:
            match_obj = (n1, n2, n3, n4, n5)
            print(f'matches case 2   : {match_obj}')
            print(r'match pattern    : (int(n1), int(n2), int(n3), int(n4), int(n5)) if [i**2 for i in range(1, 6)]' '\n')
        case [float(f1), float(f2)] if len(str(f1).split('.')[1]) == 4 and len(str(f2).split('.')[0]) == 6:
            match_obj = (f1, f2)
            print(f'matches case 3   : {match_obj}')
            print(r"match pattern    : [float(f1), float(f2)] if len(str(f1).split('.')[1]) == 4 and len(str(f2).split('.')[0]) == 6" '\n')
        case (bool(b1), bool(b2)) if b1 is not b2:
            match_obj = (b1, b2)
            print(f'matches case 4   : {match_obj}')
            print(r"match pattern    : (bool(b1), bool(b2()) if b1 is not b2" '\n')
        case {'height': float(h), 'weight': float(w), 'bmi': float(b)} if round(w / h**2, 1) == b:
            match_obj = (h, w, b)
            print(f'matches case 5   : {match_obj}')
            print(r"match pattern    : {'height': float(h), 'weight': float(w), 'bmi': float(b)} if round(w / h**2, 1) == b" '\n')
        case _:
            print(match_obj)
            print('match pattern    : _' '\n')
            pass
    return match_obj

objs = [
    ('蘇東坡', '蘇東坡'),
    ('蘇東坡', '蘇軾'),      # Q: 蘇東坡就是蘇軾，這是常識，電腦怎麼不懂？  A: 這不是AI喔
    [0, 1, 1, 2, 3, 5],      # Fibonacci sequence
    [1, 4, 9, 16, 25],       # Square Numbers
    (1.5396, 299792.458),    # Lieb's square ice constant, 光速(km/s)
    [True, False],           # toggle
    {'height': 1.68, 'weight': 66.8, 'bmi': 23.7}   # 66.8 / 1.68**2 = 21.7
]

for obj in objs:
    print(f'obj to be matched: {obj}      type: {type(obj)}')
    result = match(obj)

## <font color='LightSalmon'>番外篇：有if-elif-else能而Pattern Matching不能的嗎？</font>
* 以筆者目前的理解，萬一有如下的if-elif-else結構，Pattern Matching可能就無能為力了。
* 就算最後Pattern Matching能夠解決，大概也非常冗長難懂，直接用if-elif-else反而較易閱讀。
* 所以3.10版推出了威力強大的Pattern Matching，<font color='tomato' size='+1'><b>if-elif-else仍有其存在價值</b></font>。

In [ ]:
a = 3
b = 5
c = 14
if a - b > c:
    print("case 0")
elif a * c < 1:
    print("case 1")
elif c == 59:
    print("case 2")
elif "聯友人工資源管理公司"[5:9] == "資源管理":
    print("case 3")
elif 2 + 2 == 5:
    print("case 4")
elif False:
    print("case 5")
else:
    print("Oh, no")

## <font color='LightSalmon'>順便談一個程式邏輯</font>
* 以下利用<font color='tomato' size='+1'>randomuser.me</font>網站產生的測試用user資料來說明。
* 該網站會產生隨機user，亦可指定user id。產生的欄位蠻多的，可以作為程式的測試資料用。
* 它的1.1版和1.3版產生的user，JSON的結構和內容值會稍有不同。
* 我們取其中的dob(出生日期)欄位作為例子，利用1.1和1.3版不同結構的特性，說明一個寫code時須要注意的地方。
* 為方便觀察，就固定產生同一id(685)的user。

In [ ]:
import requests

USER_ID = 685  # 為方便觀察，就固定產生同一id的user。USER_ID全大寫是constant的慣例。


def get_user(version="1.3"):
    """Get random users"""
    url = f"https://randomuser.me/api/{version}/?results=1&seed={USER_ID}"
    response = requests.get(url)
    if response:
        return response.json()["results"][0]


user = get_user("1.1")
print("1.1版")
print(user)
print(f'出生日期：{user["dob"]}')
print(f'型別    ：{type(user["dob"])}')
print("------------------------")
print("1.3版")
user = get_user("1.3")
print(user)
print(f'出生日期：{user["dob"]}')
print(f'型別    ：{type(user["dob"])}')

## <font color='LightSalmon'>多重判斷一定要「先特殊後普通」</font>
* 多重判斷時，要把握<font color='tomato' size='+1'>越specific(特殊)者越先，越general(普通)者越後</font>這個原則。
* 萬一不小心寫反，結果就不是預期了。

### <font color='SkyBlue'>傳統if-elif-else寫法</font>

In [ ]:
from datetime import datetime

def get_age_if_specific_to_general(user):
    """
    Returns the age of parameter user.

        Parameters:
            user (requests.models.Response): user information

        Returns:
            age (int): the age of the user

        Description:
            1. 使用Python傳統的if-elif-else語法。
            2. if-elif順序是先general(普遍)後specific(特殊)。
    """
    if "age" in user["dob"]:  # specific(1.3版才有'age'欄位)
        return user["dob"]["age"]
    elif "dob" in user:  # general(1.1到1.3版都有'dob'欄位)
        now = datetime.now()
        dob_date = datetime.strptime(user["dob"], "%Y-%m-%d %H:%M:%S")
        return now.year - dob_date.year
    else:
        return None


def get_age_if_general_to_specific(user):
    """
    Returns the age of parameter user.

        Parameters:
            user (requests.models.Response): user information

        Returns:
            age (int): the age of the user

        Description:
            1. 使用Python傳統的if-elif-else語法。
            2. if-elif順序是先general(普遍)後specific(特殊)。
    """
    if "dob" in user:  # general(1.1到1.3版都有'dob'欄位)
        now = datetime.now()
        dob_date = datetime.strptime(user["dob"], "%Y-%m-%d %H:%M:%S")
        return now.year - dob_date.year
    elif "age" in user["dob"]:  # specific(1.3版才有'age'欄位)
        return user["dob"]["age"]
    else:
        return None

### <font color='SkyBlue'>Pattern Matching寫法</font>

In [ ]:
def get_age_match_specific_to_general(user):
    '''
    Returns the age of parameter user.

        Parameters:
            user (requests.models.Response): user information

        Returns:
            age (int): the age of the user

        Description:
            1. 使用Python 3.10新增的structural Pattern Match技術。
            2. 前兩case's順序是先specific(特殊)後general(普遍)。
    '''
    match user:
        case {'dob': {'age': age}}:    # specific(1.3版才有'age'欄位)
            return age
        case {'dob': dob}:             # general(1.1到1.3版都有'dob'欄位)
            now = datetime.now()
            dob_date = datetime.strptime(dob, '%Y-%m-%d %H:%M:%S')
            return now.year - dob_date.year
        case _:
            return None

def get_age_match_general_to_specific(user):
    '''
    Returns the age of parameter user.

        Parameters:
            user (requests.models.Response): user information

        Returns:
            age (int): the age of the user

        Description:
            1. 使用Python 3.10新增的structural Pattern Match技術。
            2. 前兩case's順序是先general(普遍)後specific(特殊)。
    '''
    match user:
        case {'dob': dob}:             # general(1.1到1.3版都有'dob'欄位)
            now = datetime.now()
            dob_date = datetime.strptime(dob, '%Y-%m-%d %H:%M:%S')
            return now.year - dob_date.year
        case {'dob': {'age': age}}:    # specific(1.3版才有'age'欄位)
            return age
        case _:
            return None

In [ ]:
while True:
    try:
        version = input(
            "Input version(1.1 or 1.3)   999 exit: "
        ).strip()  # 請輸入1.1或1.3，或999結束。
        if version == "999":
            print("The End.")
            break
        else:
            print(f"version: {version}\n")
            user = get_user(version=version)
            # user["dob"]
            age_if_specific_to_general = get_age_if_specific_to_general(user)
            print(f"age_if_specific_to_general: {age_if_specific_to_general}")
            age_if_general_to_specific = get_age_if_general_to_specific(user)
            print(f"age_if_general_to_specific: {age_if_general_to_specific}")
            age_match_specific_to_general = get_age_match_specific_to_general(user)
            print(f"age_match_specific_to_general: {age_match_specific_to_general}")
            age_match_general_to_specific = get_age_match_general_to_specific(user)
            print(f"age_match_general_to_specific: {age_match_general_to_specific}")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        print()

## <font color='LightSalmon'>「先普通後特殊」的錯誤示範</font>
* 下列的code如輸入50，您認為理應印出甚麼？
* 實際上會印出甚麼？
* 麻煩修正。

In [ ]:
# 從最general到最specific的寫法錯誤。
while True:
    age = int(input("Input age (999 exit):"))
    if age == 999:
        print("Game Over.")
        break
    elif age >= 12:  # 最general
        print("可看輔級電影。")
    elif age >= 20:
        print("可看輔級電影和投票。")
    elif age >= 40:
        print("可看輔級電影、投票和選總統。")
    elif age >= 65:  # 最specific
        print("可看輔級電影、投票、選總統和退休。")
    else:
        print("年齡太小，長大再說。")

## <font color='LightSalmon'>「先普通後特殊」錯誤示範的參考修訂</font>

In [ ]:
# 從最specific到最general的寫法才正確。
while True:
    age = int(input("Input age (999 exit):"))
    if age == 999:
        print("Game Over.")
        break
    elif age >= 65:  # 最specific
        print("可看輔級電影、投票、選總統和退休。")
    elif age >= 40:
        print("可看輔級電影、投票和選總統。")
    elif age >= 20:
        print("可看輔級電影和投票。")
    elif age >= 12:  # 最general
        print("可看輔級電影。")
    else:
        print("年齡太小，長大再說。")